# Overview

1. We'll first introduce Spark both in batch mode and streaming in [Spark Datasets and Structured Streaming](Spark_Datasets_and_Structured_Streaming.ipynb).
1. Second, we'll do a deep dive into [Spark Structured Streaming](Spark_Structured_Streaming.ipynb) exploring the API.
1. Next, we'll go under the hood, looking at different improvements, and comparing Spark and Spark Streaming in [DStream Comparison](DStream_Comparison.ipynb).
1. Finally, we'll look at how to build [Standalone Spark Streaming Applications](Standalone_Spark_Streaming_Applications.ipynb)


## About The Instructor
Michael Li is the founder of [The Data Incubator](https://www.thedataincubator.com), an 8 week fellowship to help PhDs and postdocs transition from  academia into industry.  Previously, he headed monetization data science at Foursquare and has worked at Google, Andreessen Horowitz, J.P. Morgan, and D.E. Shaw.  He is a regular contributor to [Harvard Business Review](https://hbr.org/search?term=michael+li), [O'Reilly](https://www.oreilly.com/people/76a5b-michael-li), [Wall Street Journal](http://www.wsj.com/articles/calculus-is-so-last-century-1457132991), [VentureBeat](http://venturebeat.com/author/michael-li-the-data-incubator/), and [Fast Company](http://www.fastcompany.com/3049788/how-to-get-past-buzzwords-and-make-better-hires).  Michael earned his PhD at Princeton and was a Marshall Scholar in Cambridge.


## About The Data Incubator

[The Data Incubator](https://www.thedataincubator.com) is a data science education company based in NYC, DC, and SF with both corporate training and recruiting offerings. For [data science corporate training](https://www.thedataincubator.com/training.html), we offer customized, in-house corporate training solutions in data and analytics (including additional Spark offerings!). For [data science hiring](https://www.thedataincubator.com/hiring.html), we run a [free 8 week fellowship](https://www.thedataincubator.com/fellowship.html) training PhDs to become data scientists. The fellowship selects 2% of its 2000+ quarterly applicants and is free for fellows. Hiring companies only pay a recruiting fee only if they successfully hire. You can read about us on VentureBeat or The Next Web, or read about our alumni at [LinkedIn](http://blog.thedataincubator.com/2016/05/alumni-spotlight-xia-hong/), [Capital One](http://blog.thedataincubator.com/2015/08/columbia-astrophysics-postdoc-moves-to-capital-one-labs-alumni-spotlight-on-brian-farris/), [Palantir](http://blog.thedataincubator.com/2015/02/moving-to-palantir-from-mathematics-alumni-spotlight-on-justin-bush/), [Amazon](http://blog.thedataincubator.com/2016/06/david-wallace/) or the [NYTimes](http://blog.thedataincubator.com/2015/02/alumni-spotlight-dorian-goldman-using-a-pure-math-background-to-solve-problems-for-the-new-york-times/).

<img src="images/logo-text.jpg" width="20%"/>